<a href="https://colab.research.google.com/github/learningmatter-mit/uvvisml/blob/main/uvvisml_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
!chmod +x Miniconda3-py37_4.8.3-Linux-x86_64.sh
!bash ./Miniconda3-py37_4.8.3-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y -c conda-forge rdkit==2020.09.2
!conda install -q -y -c conda-forge chemprop
!git clone https://github.com/learningmatter-mit/uvvisml

import os
import sys
import pandas as pd
sys.path.append('/usr/local/lib/python3.7/site-packages/')
import chemprop

os.chdir('uvvisml/uvvisml')

!bash get_model_files.sh

--2022-02-05 05:19:39--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.3-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

PREFIX=/usr/local
Unpacking payload ...
Solving environment: | / - \ | / - \ | / 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - conda-forge/linux-64::rdkit==2020.09.2=py37h713bca6_0
  - conda-forge/noarch::sphinxcontrib-qthelp==1.0.3=py_0
  - defaults/noarch::urllib3==1.26.6=pyhd3eb1b0_1
  - defaults/linux-64::python==3.7.7=hcff3b4d_5
  - defaults/linux-64::ncurses==6.2=he6710b0_1
  - conda-forge/noarch::hyperopt==0.2.5=pyh9f0ad1d_0
  - conda-forge/linux-64::pcre==8.45=

# Data

In [3]:
test_file = 'data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv'
df = pd.read_csv(test_file)
df

,smiles,solvent,peakwavs_max
0,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,C1CCCCC1,376.0
1,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCOC(C)=O,392.0
2,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CC#N,396.0
3,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCO,400.0
4,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,OCC(O)CO,413.0
...,...,...,...
1705,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,C[N+](=O)[O-],424.0
1706,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,CS(C)=O,432.0
1707,COc1cc(C)c(-c2cc(-c3c(C)cc(OC)cc3C)c3ccc4c(-c5...,ClCCl,367.0
1708,N#Cc1c(N2CCCCC2)cc(-c2cccc3ccccc23)c2c1-c1cccc...,C1CCOC1,358.0


# Make Predictions

## Predict experimental peak with model trained on combined training set

**Equivalent to command line:**

python uvvisml/predict.py --test_file uvvisml/data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv --property absorption_peak_nm_expt --method chemprop --preds_file test_preds.csv

In [4]:
arguments = [
  '--test_path', test_file,
  '--preds_path', '/dev/null',
  '--checkpoint_dir', 'models/lambda_max_abs/chemprop/combined/production/fold_0',
  '--number_of_molecules', '2',
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

preds = [x[0] for x in preds]
df['peakwavs_max_pred'] = preds
df

Loading training args
Setting molecule featurization parameters to default.
Loading data


1710it [00:00, 126450.28it/s]
100%|██████████| 1710/1710 [00:00<00:00, 221927.71it/s]

Validating SMILES



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test size = 1,710
Predicting with an ensemble of 5 models


...


 80%|████████  | 4/5 [00:30<00:07,  7.17s/it]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "encoder.encoder.1.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.1.W_i.weight".
Loading pretrained parameter "encoder.encoder.1.W_h.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Loading pretrained parameter "ffn.7.weight".
Loading pretrained parameter "ffn.7.bias".



100%|██████████| 5/5 [00:37<00:00,  7.41s/it]


Saving predictions to /dev/null
Elapsed time = 0:00:38


,smiles,solvent,peakwavs_max,peakwavs_max_pred
0,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,C1CCCCC1,376.0,378.089791
1,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCOC(C)=O,392.0,388.387075
2,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CC#N,396.0,394.557472
3,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCO,400.0,400.817724
4,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,OCC(O)CO,413.0,410.902339
...,...,...,...,...
1705,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,C[N+](=O)[O-],424.0,425.084342
1706,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,CS(C)=O,432.0,428.209718
1707,COc1cc(C)c(-c2cc(-c3c(C)cc(OC)cc3C)c3ccc4c(-c5...,ClCCl,367.0,370.200150
1708,N#Cc1c(N2CCCCC2)cc(-c2cccc3ccccc23)c2c1-c1cccc...,C1CCOC1,358.0,355.987562


## Predict TDDFT peak in vacuum

**Equivalent to command line:**

python uvvisml/predict.py --test_file uvvisml/data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv --property vertical_excitation_eV_tddft --method chemprop --preds_file test_preds.csv

In [5]:
arguments = [
  '--test_path', test_file,
  '--preds_path', '/dev/null',
  '--checkpoint_dir', 'models/lambda_max_abs_wb97xd3/chemprop/all_wb97xd3/production/fold_0',
  '--number_of_molecules', '1',
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

preds = [x[0] for x in preds] # predictions are in eV
df['peakwavs_max_pred'] = preds
df['peakwavs_max_pred'] = 1240/df['peakwavs_max_pred'] # convert from eV to nm
df

Loading training args
Setting molecule featurization parameters to default.
Loading data


1710it [00:00, 80922.70it/s]
100%|██████████| 1710/1710 [00:00<00:00, 161116.45it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validating SMILES
Test size = 1,710
Predicting with an ensemble of 5 models


  0%|          | 0/5 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".


...


 80%|████████  | 4/5 [00:32<00:08,  8.21s/it]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".



100%|██████████| 5/5 [00:41<00:00,  8.28s/it]


Saving predictions to /dev/null
Elapsed time = 0:00:42


,smiles,solvent,peakwavs_max,peakwavs_max_pred
0,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,C1CCCCC1,376.0,309.772465
1,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCOC(C)=O,392.0,309.772465
2,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CC#N,396.0,309.772465
3,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCO,400.0,309.772465
4,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,OCC(O)CO,413.0,309.772465
...,...,...,...,...
1705,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,C[N+](=O)[O-],424.0,346.755560
1706,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,CS(C)=O,432.0,346.755560
1707,COc1cc(C)c(-c2cc(-c3c(C)cc(OC)cc3C)c3ccc4c(-c5...,ClCCl,367.0,318.639569
1708,N#Cc1c(N2CCCCC2)cc(-c2cccc3ccccc23)c2c1-c1cccc...,C1CCOC1,358.0,310.870571


## Predict experimental peak with model trained on Deep4Chem training set

**Equivalent to command line:**

python uvvisml/predict.py --test_file uvvisml/data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv --property absorption_peak_nm_expt --method chemprop --preds_file test_preds.csv --train_dataset deep4chem

In [6]:
arguments = [
  '--test_path', test_file,
  '--preds_path', '/dev/null',
  '--checkpoint_dir', 'models/lambda_max_abs/chemprop/deep4chem/production/fold_0',
  '--number_of_molecules', '2',
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

preds = [x[0] for x in preds]
df['peakwavs_max_pred'] = preds
df

Loading training args
Setting molecule featurization parameters to default.
Loading data


1710it [00:00, 141559.62it/s]
100%|██████████| 1710/1710 [00:00<00:00, 153911.16it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validating SMILES
Test size = 1,710
Predicting with an ensemble of 5 models


  0%|          | 0/5 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "encoder.encoder.1.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.1.W_i.weight".
Loading pretrained parameter "encoder.encoder.1.W_h.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".


...


 80%|████████  | 4/5 [00:52<00:13, 13.10s/it]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "encoder.encoder.1.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.1.W_i.weight".
Loading pretrained parameter "encoder.encoder.1.W_h.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".



100%|██████████| 5/5 [01:05<00:00, 13.18s/it]

Saving predictions to /dev/null
Elapsed time = 0:01:06


,smiles,solvent,peakwavs_max,peakwavs_max_pred
0,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,C1CCCCC1,376.0,382.903437
1,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCOC(C)=O,392.0,395.478472
2,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CC#N,396.0,400.821401
3,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCO,400.0,418.106349
4,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,OCC(O)CO,413.0,426.706045
...,...,...,...,...
1705,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,C[N+](=O)[O-],424.0,474.872657
1706,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,CS(C)=O,432.0,466.555822
1707,COc1cc(C)c(-c2cc(-c3c(C)cc(OC)cc3C)c3ccc4c(-c5...,ClCCl,367.0,365.403474
1708,N#Cc1c(N2CCCCC2)cc(-c2cccc3ccccc23)c2c1-c1cccc...,C1CCOC1,358.0,349.835862


## Predict experimental peak with multi-fidelity model

**Equivalent to command line:**

python uvvisml/predict.py --test_file uvvisml/data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv --property absorption_peak_nm_expt --method chemprop_tddft --preds_file test_preds.csv

In [7]:
# TDDFT Predictions
arguments = [
  '--test_path', test_file,
  '--preds_path', 'test_tddft_preds.csv',
  '--checkpoint_dir', 'models/lambda_max_abs_wb97xd3/chemprop/all_wb97xd3/production/fold_0',
  '--number_of_molecules', '1',
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
_ = chemprop.train.make_predictions(args=args)

# Convert Predictions to Features File
!python models/tddft_to_features_file.py

# Experimental Predictions
arguments = [
  '--test_path', test_file,
  '--preds_path', '/dev/null',
  '--checkpoint_dir', 'models/lambda_max_abs/chemprop_tddft/combined/production/fold_0',
  '--number_of_molecules', '2',
  '--features_path', 'features_test.csv'
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
preds = chemprop.train.make_predictions(args=args)

preds = [x[0] for x in preds]
df['peakwavs_max_pred'] = preds
df

Loading training args
Setting molecule featurization parameters to default.
Loading data


1710it [00:00, 134827.05it/s]
100%|██████████| 1710/1710 [00:00<00:00, 136684.77it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validating SMILES
Test size = 1,710
Predicting with an ensemble of 5 models


  0%|          | 0/5 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".


...


 80%|████████  | 4/5 [07:06<01:46, 106.39s/it]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "encoder.encoder.1.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.1.W_i.weight".
Loading pretrained parameter "encoder.encoder.1.W_h.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Loading pretrained parameter "ffn.7.weight".
Loading pretrained parameter "ffn.7.bias".



100%|██████████| 5/5 [08:51<00:00, 106.35s/it]


Saving predictions to /dev/null
Elapsed time = 0:08:52


,smiles,solvent,peakwavs_max,peakwavs_max_pred
0,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,C1CCCCC1,376.0,375.545257
1,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCOC(C)=O,392.0,390.993980
2,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CC#N,396.0,397.488817
3,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCO,400.0,400.081324
4,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,OCC(O)CO,413.0,412.967337
...,...,...,...,...
1705,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,C[N+](=O)[O-],424.0,424.124035
1706,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,CS(C)=O,432.0,428.538180
1707,COc1cc(C)c(-c2cc(-c3c(C)cc(OC)cc3C)c3ccc4c(-c5...,ClCCl,367.0,355.781207
1708,N#Cc1c(N2CCCCC2)cc(-c2cccc3ccccc23)c2c1-c1cccc...,C1CCOC1,358.0,358.098561


## Predict experimental peak with model trained on combined training set (with ensemble variance)

**Equivalent to command line:**

python uvvisml/predict.py --test_file uvvisml/data/splits/lambda_max_abs/deep4chem/group_by_smiles/smiles_target_test.csv --property absorption_peak_nm_expt --method chemprop --preds_file test_preds.csv

In [8]:
arguments = [
  '--test_path', test_file,
  '--preds_path', 'test_preds.csv',
  '--checkpoint_dir', 'models/lambda_max_abs/chemprop/combined/production/fold_0',
  '--number_of_molecules', '2',
  '--ensemble_variance',
  #'--gpu', '0'
]

args = chemprop.args.PredictArgs().parse_args(arguments)
_ = chemprop.train.make_predictions(args=args)

df = pd.read_csv('test_preds.csv')
df

Loading training args
Setting molecule featurization parameters to default.
Loading data


1710it [00:00, 129826.41it/s]
100%|██████████| 1710/1710 [00:00<00:00, 108043.62it/s]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Validating SMILES
Test size = 1,710
Predicting with an ensemble of 5 models


  0%|          | 0/5 [00:00<?, ?it/s]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "encoder.encoder.1.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.1.W_i.weight".
Loading pretrained parameter "encoder.encoder.1.W_h.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Loading pretrained parameter "ffn.7.weight".
Loading pretrained parameter "ffn.7.bias".


...


 80%|████████  | 4/5 [00:26<00:06,  6.77s/it]

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "encoder.encoder.1.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.1.W_i.weight".
Loading pretrained parameter "encoder.encoder.1.W_h.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.weight".
Loading pretrained parameter "encoder.encoder.1.W_o.bias".
Loading pretrained parameter "ffn.1.weight".
Loading pretrained parameter "ffn.1.bias".
Loading pretrained parameter "ffn.4.weight".
Loading pretrained parameter "ffn.4.bias".
Loading pretrained parameter "ffn.7.weight".
Loading pretrained parameter "ffn.7.bias".



100%|██████████| 5/5 [00:33<00:00,  6.75s/it]


Saving predictions to test_preds.csv
Elapsed time = 0:00:34


,smiles,solvent,peakwavs_max,peakwavs_max_epi_unc
0,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,C1CCCCC1,378.089791,5.984684
1,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCOC(C)=O,388.387075,3.289096
2,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CC#N,394.557472,6.647632
3,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,CCO,400.817724,7.351677
4,CCN(CC)c1ccc2c(C(F)(F)F)cc(=O)oc2c1,OCC(O)CO,410.902339,5.109207
...,...,...,...,...
1705,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,C[N+](=O)[O-],425.084342,8.989382
1706,c1cc2c3ccc[n+]4cccc(c5ccc[n+](c1)c25)c34,CS(C)=O,428.209718,9.853369
1707,COc1cc(C)c(-c2cc(-c3c(C)cc(OC)cc3C)c3ccc4c(-c5...,ClCCl,370.200150,128.172913
1708,N#Cc1c(N2CCCCC2)cc(-c2cccc3ccccc23)c2c1-c1cccc...,C1CCOC1,355.987562,2.114018
